# XML Data Ingestion with LakeLogic 

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LineageLogic/LakeLogic/blob/main/examples/03_data_sources/files/xml/xml_example.ipynb)
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/LineageLogic/LakeLogic/blob/main/examples/03_data_sources/files/xml/xml_example.ipynb)

## Business Scenario
Your organization receives employee data exports from legacy HR systems in XML format. These files contain employee records with critical information like emails, salaries, and employment status.

Common data quality issues:
- Invalid email formats (missing @ symbol)
- Negative or zero salaries (data entry errors)
- Inactive employees included in active reports

## Value Proposition
- **Native XML Support**: No manual parsing required - point LakeLogic directly at `.xml` files
- **Engine Agnostic**: Works with Polars, Pandas, DuckDB, or Spark
- **Automatic Validation**: Catch data quality issues before they reach your warehouse
- **Clear Error Tracking**: Every quarantined record includes specific failure reasons

## 1. Setup
First, we'll install LakeLogic (if needed) and import the necessary libraries.

In [ ]:
from pathlib import Path
from lakelogic import DataProcessor

BASE = Path.cwd()
contract_path = BASE / "xml_contract.yaml"
if not contract_path.exists():
    candidate = BASE / "examples" / "03_data_sources" / "files" / "xml" / "xml_contract.yaml"
    if candidate.exists():
        BASE = candidate.parent
        contract_path = candidate

xml_path = BASE / "data" / "employees.xml"
result = None

def preview_frame(df, title=None, limit=5):
    if title:
        print(title)
    if df is None:
        print("<empty>")
        return
    if hasattr(df, "head"):
        try:
            print(df.head(limit))
            return
        except Exception:
            pass
    if hasattr(df, "limit"):
        try:
            sample = df.limit(limit)
            print(sample)
            return
        except Exception:
            pass
    print(df)


## 2. Preview the XML Data
Let's take a quick look at the raw XML file to see what we're working with.

In [ ]:
if not xml_path.exists():
    raise FileNotFoundError(f"Missing XML file: {xml_path.resolve()}")

with open(xml_path, "r", encoding="utf-8") as handle:
    content = handle.read(800)
    print(content)
    print("...")


## 3. Run LakeLogic with XML Source
Now we'll use the Data Contract to ingest, validate, and transform the XML data.

In [ ]:
processor = DataProcessor(contract=contract_path)
try:
    result = processor.run_source(xml_path)
except Exception as exc:
    print(f"XML ingestion failed: {exc}")
    result = None

if result is not None:
    print(result)
    print(f"Raw: {len(result.raw)} | Good: {len(result.good)} | Bad: {len(result.bad)}")


## 4. Inspect Raw XML Data
Let's see what the data looks like after XML parsing (before validation).

In [ ]:
if result is None:
    print("No results to display.")
else:
    preview_frame(result.good, "VALIDATED DATA (sample):")
    preview_frame(result.bad, "QUARANTINED DATA (sample):")


## 5. Inspect Validated Records (Silver Layer)
These records passed all validation rules and transformations.

## 6. Inspect Quarantined Records
These records failed validation - let's see why.

## 7. Data Quality Metrics
Let's calculate some key metrics about our data quality.

## 8. Try Different Engines
LakeLogic supports multiple engines. Let's try switching engines to see the flexibility.

## Summary

In this tutorial, we demonstrated:

1.  **Native XML Ingestion**: No manual parsing - LakeLogic automatically detected and read the `.xml` file
2.  **Automatic Validation**: Email formats, salary ranges, and employment status were validated
3.  **Clear Error Tracking**: Each quarantined record includes specific reasons for failure
4.  **Data Enrichment**: Added `seniority_level` based on hire date
5.  **Engine Flexibility**: Same contract works with Polars, Pandas, DuckDB, or Spark

### Next Steps

- **Modify the contract**: Try adding your own validation rules to `xml_contract.yaml`
- **Change the data**: Edit `employees.xml` to test different validation scenarios
- **Materialize results**: Use LakeLogic's materialization feature to persist validated data
- **Production deployment**: Integrate this into your data pipeline for automated XML ingestion